In [ ]:
import os
import pandas as pd
from collections import Counter
import numpy as np
import random
import json
import networkx as nx
import pickle

#supernoder files:

#вариант, который считает РАЗНЫЕ типы мотивов одного и того же размера
from SuperNoder_diff_types.manager import Manager as Manager_types 

#вариант, который считает все мотивы одного размера вместе
from SuperNoder.manager import Manager as Manager

#!pip install ipyparallel

# you should "ipcluster start -n 4" on your local machine
**instead of 4 you should write the number of engines on your computer**

In [ ]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

**Read datasets**

In [ ]:
for root, dirs, files in os.walk(r"C:\Users\anpolol\Desktop\DifusionPredictionUsingMotifs\Data\reply_networks"):
    datasets_names = files
    
#in every graph there are 11 networks, all_nets consists of all nets  
all_nets = dict()
for dataset in datasets_names:
    if dataset not in all_nets:
        all_nets[dataset] = []
        month_nets = json.load(open(r"C:\Users\anpolol\Desktop\DifusionPredictionUsingMotifs\Data\reply_networks\\"+str(dataset)))
        all_nets[dataset]=all_nets[dataset] + month_nets
        
path=r'Data/'

**Запустите или клетку ниже, чтоб отфлитровать данные (дольше), или клетку ниже через одну, чтоб загрузить отфильтрованные данные (быстрее)**

`type(graphs) = list of tuples: [(str(name),networkx(graph))]`

In [ ]:
#фильтрация по размеру исходных графов
def func(inp):
        from collections import Counter
        import networkx as nx
        dataset,values = inp
        name = dataset.split('.')
        graphs=[]
        for i,net in enumerate(values):

            t=list(net.values()) #extracting nodes from json files
            nodes = (Counter([item for sublist in t for item in sublist] + list(net.keys())))

            map_nodes=dict(zip(nodes, list(range(len(nodes))))) #нумеруем ники

            g = nx.Graph()
            for node in range(len(nodes)):
                g.add_node(node,label='Motif')

            for node in net:
                for neigh in net[node]:
                    g.add_edge((map_nodes[node]),(map_nodes[neigh])) #first node replied to the second   
            for j,k in enumerate(nx.connected_components(g)):
                if (len(k)>300) and (len(k)<3000):
                    graphs.append((name[0]+'.'+str(i)+'.'+str(j),g.subgraph(k))) #i - месяц, j - номер кмпоненты связности
            return graphs
        
res = dview.map(func, list(all_nets.items())).get()

graphs=[]
for i in res:
    graphs+=i

In [ ]:
#загрузка отфильтрованных данных
from functools import reduce 
import pickle

with open('all_graphs.pickle', 'rb') as f:
    graphs = pickle.load(f)

**Характеристика отфильтрованных графов**
- число узлов 
- плотность
- к-т кластеризации (общий, для графа)
- ассортативность (по степени)

In [ ]:
#конвертируем список кортежей в словарь для более быстрого счета
new_names = list(map(lambda x: x.split('.')[0], list(zip(*graphs))[0]))
new_graphs = list(zip(new_names, list(zip(*graphs))[1]))

graphs_dict=dict()
for name,g in new_graphs:
    if name not in graphs_dict:
        graphs_dict[name]=[]
    graphs_dict[name].append(g)

_На данном этапе можно отобрать нужное количество графов, сейчас grahs_dict содержит все компоненты связности размером от 300 до 3000 каждой из тем. Тема - ключ, список графов - значения словаря_

In [ ]:
# Характеристика графов по темам
table_names=[]
table_NN_mean=[]
table_CC_mean=[]
table_D_mean=[]
table_DAC_mean=[]

table_NN_std=[]
table_CC_std=[]
table_D_std=[]
table_DAC_std=[]


for k in graphs_dict:
    number_of_nodes = []
    clustering_coef=[]
    density=[]
    degree_assort=[]
    graphs=[]

    table_names.append(k)
    
    for gr in graphs_dict[k]:
            #число узлов 
            number_of_nodes.append(gr.number_of_nodes())
            #коэффициенты кластеризации ищутся следующим образом:
            clustering_coef.append(np.mean(list(nx.clustering(gr).values())))
            #плотность
            density.append(nx.density(gr))
            #ассортативность
            degree_assort.append(nx.degree_assortativity_coefficient(gr))
    table_NN_mean.append(np.mean(np.array(number_of_nodes)))
    table_CC_mean.append(np.mean(np.array(clustering_coef)))
    table_D_mean.append(np.mean(np.array(density)))
    table_DAC_mean.append(np.mean(np.array(degree_assort)))
    table_NN_std.append(np.std(np.array(number_of_nodes)))
    table_CC_std.append(np.std(np.array(clustering_coef)))
    table_D_std.append(np.std(np.array(density)))
    table_DAC_std.append(np.std(np.array(degree_assort)))

In [ ]:
mean_df = pd.DataFrame({'Name of dataset' : table_names,
                                'Number of nodes (NN)' :table_NN_mean,
                                'Clustering coefficient (CC)': table_CC_mean,'Density(D)':table_D_mean, 'Degree assortativity coefficient(DAC)':table_DAC_mean})
std_df = pd.DataFrame({'Name of dataset' : table_names,
                                'Number of nodes (NN)' :table_NN_std,
                                'Clustering coefficient (CC)': table_CC_std,'Density(D)':table_D_std, 'Degree assortativity coefficient(DAC)':table_DAC_std})

# **Motif counter - SuperNoder** 

**arguments for SuperNoder:**

-n,  --nodes-file <filename> MANDATORY The list of nodes. Node id and label for each row separated by a space\
-e,  --edges-file <filename> MANDATORY The list of edges. One edge for each row.\
-m,  --method <method> OPTIONAL The heuristic to use in order to maximize motifs. DEFAULT: h1 \
-tn, --type-of-network <type> OPTIONAL 	The type of network. It can be chosen from [direct, undirect]. DEFAULT: undirect\ 
-th, --threshold <threshold> OPTIONAL The threshold to hold over-represented motifs.\
-ms, --motif-size <size> OPTIONAL The size of motifs. It must be greater or equal to 3. DEFAULT: 3\
-h1tr, --h1-times-repetition <times> OPTIONAL \tThe number of repetition of h1. DEFAULT: 1\
-ss, --samples-size <sample_size> OPTIONAL The size of samples for heuristics h4 and h5. DEFAULT: 100\

In [ ]:
def find_disjoint_motifs(g):
    from SuperNoder.manager import Manager as Manager_types 
    arg_tn='undirect'
    arg_th = '1' 
    arg_m = 'h1'
    arg_ss = '100'
    arg_h1tr = 1 
    distributions = {}
    distributions_disjoint = {}
    for arg_ms in range(3,5):
        argv=[' ','-g',g[1],'-th',arg_th,'-ms',arg_ms,'-m',arg_m,\
      '-h1tr',arg_h1tr,'-ss',arg_ss]
        m = Manager_types(argv)
        distribution_f1 = m.run()
        distribution_disjoint = m.disjoint_finder()
        distributions = dict(list(distributions.items()) + list(distribution_f1.items()))
        distributions_disjoint = dict(list(distributions_disjoint.items()) + list(distribution_disjoint.items()))
    return  g[0],distributions_disjoint,distributions#первое значение словаря - тип мотива или размер мотива, второе значение - количество таких мотивов в графе

In [ ]:
# here is a parallelization
res = dview.map(find_disjoint_motifs,graphs[:2]).get()

In [ ]:
#distributions of motifs for every net:
Motif_f1 = dict()
Motif_f3 = dict()
for (name,f3,f1) in res:
    Motif_f3[name] = f3
    Motif_f1[name] = f1

**составляем матрицу входных данных таким образом, чтоб она была одной и той же длины для любого датасета**

In [ ]:
names_of_all_motifs=[] #list of all motif types in all datasets
for dataset in Motif_f3:
    for name_of_motif in Motif_f3[dataset]:
        if name_of_motif not in names_of_all_motifs:
            names_of_all_motifs.append(str(name_of_motif))
names_of_all_motifs=sorted(names_of_all_motifs)

In [ ]:
X = np.zeros((len(args_e),len(names_of_all_motifs)))
for i,d in enumerate(args_e):
    for k,m in enumerate(names_of_all_motifs):
        if m in Motif_f3[d]:
            X[i][k] = Motif_f3[d][m]/sum(Motif_f3[d].values())  

# Sampling

In [ ]:
from littleballoffur import BreadthFirstSearchSampler, \
                            PageRankBasedSampler, \
                            CommunityStructureExpansionSampler, \
                            MetropolisHastingsRandomWalkSampler, \
                            RandomWalkSampler, \
                            RandomNodeNeighborSampler, \
                            CommonNeighborAwareRandomWalkSampler
methods = [
           #random node sampling
           DegreeBasedSampler,
           PageRankBasedSampler,

           #Random Edge Sampling
           RandomEdgeSampler,
    
           SnowBallSampler,
           ForestFireSampler,
           CommunityStructureExpansionSampler,
           ShortestPathSampler,
            #Random-Walks Dased
           RandomWalkSampler,
           RandomWalkWithJumpSampler,
           MetropolisHastingsRandomWalkSampler,
           NonBackTrackingRandomWalkSampler,
           CirculatedNeighborsRandomWalkSampler,
           CommonNeighborAwareRandomWalkSampler,
           LoopErasedRandomWalkSampler
          ]

In [ ]:
from sklearn.metrics import mean_squared_error 
arg_tn='undirect'
arg_th = '1' 
arg_m = 'h1'
arg_ss = '100'
arg_h1tr = 1 

graph_samples = []
MSE_samples = []
graph = graphs[0]
distrubutions_full=dict()
for arg_ms in range(3,5):
    argv=[' ','-g',graph[1],'-th',arg_th,'-ms',arg_ms,'-m',arg_m,'-h1tr',arg_h1tr,'-ss',arg_ss]      
    m = Manager_types(argv)
    motif_full_f1 = m.run()
    distrubutions_full = dict(list(distrubutions_full.items()) + list(motif_full_f1.items()))
distrubutions_full

In [ ]:
def motifs_to_vec(motifs1, motifs2):
    names_of_all_motifs=[] #list of all motif types in all datasets
    for name_of_motif in motifs1:
        if name_of_motif not in names_of_all_motifs:
            names_of_all_motifs.append(str(name_of_motif))
    for name_of_motif in motifs2:
        if name_of_motif not in names_of_all_motifs:
            names_of_all_motifs.append(str(name_of_motif))
    names_of_all_motifs=sorted(names_of_all_motifs)
    
    new_motifs = np.zeros(2,len(names_of_all_motifs))
    for k,m in enumerate(names_of_all_motifs):
        if m in motifs1:
            new_motifs[0][k] = motifs1[m]
            new_motifs[1][k] = motifs2[m]
                
    return new_motifs[0], new_motifs[1]

In [ ]:
number_of_nodes = int(0.5*graph[1].number_of_nodes()) 
for method in methods:
    sampler = method(number_of_nodes)
    sample = sampler.sample(graph[1])
    distribution_sample =dict()
    for arg_ms in range(3,5):
        argv=[' ','-g',graph[1],'-th',arg_th,'-ms',arg_ms,'-m',arg_m,'-h1tr',arg_h1tr,'-ss',arg_ss]      
        m = Manager_types(argv)
        motif_sample_f1 = m.run()
        distribution_sample = dict(list(distributions_sample.items()) + list(motif_sample_f1.items()))
    MSE.append(mean_squared_error(motifs_full,motifs_sample))
print('MSE: ', MSE)